In [14]:
import pandas as pd, os, json
import numpy as np

In [ ]:
ROOT_FOLDER = "<path>/Experiments"

In [ ]:
def get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay):

    if task == "prostate":
        if ema_decay == 0.99:
            path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_OctreeNCASegmentation")
        else:
            path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCASegmentation")
    elif task == "peso":
        path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCA2DSegmentation")
    elif task == "cholec":
        #path = os.path.join(ROOT_FOLDER, f"{task}fAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCASegmentation")
        path = os.path.join(ROOT_FOLDER, f"{task}fFixAbl_{norm}_{steps}_{alpha}_{model}_{batch}_{dice_weight}_{ema_decay}_OctreeNCASegmentation")
        


    mem_in_gb = json.load(open(os.path.join(path, "mem_allocation.json")))["byte"] / 1024**3

    if not os.path.exists(os.path.join(path, "eval", "standard.csv")):
        return {
            "D": None,
            "M": mem_in_gb
        }
    seg_results = pd.read_csv(os.path.join(path, "eval", "standard.csv"), sep="\t")
    
    if task == 'cholec':
        nan_template = pd.read_csv(f"<path>/Experiments/cholec_sam_Sam3DSegmentation/eval/standard.csv", sep='\t')
        nan_template.set_index('Unnamed: 0', inplace=True)
        seg_results.set_index('Unnamed: 0', inplace=True)
        seg_results[nan_template.isna()] = np.nan


    #mean_dice = seg_results.mean(numeric_only=True)["DiceScore/5"]

    mean_dice = seg_results.mean(numeric_only=True)
    mean_dice = mean_dice.filter(like="DiceScore").mean()

    return{
        "D": mean_dice,
        "M": mem_in_gb
    }

In [17]:
def preprocess(x):
    # find best 'D'
    ds = [e["D"] for e in x]
    ds_sorted = [e["D"] for e in x]
    ds_sorted.sort(reverse=True)
    best_d, sbest_d = ds_sorted[0], ds_sorted[1]
    best_d_idx, sbest_d_idx = ds.index(best_d), ds.index(sbest_d)
    
    # find best 'M'
    ms = [e["M"] for e in x]
    ms_sorted = [e["M"] for e in x]
    ms_sorted.sort()
    best_m, sbest_m = ms_sorted[0], ms_sorted[1]
    best_m_idx, sbest_m_idx = ms.index(best_m), ms.index(sbest_m)
    if ms.count(best_m) > 1:
        best_m_idx = None
    if ms.count(sbest_m) > 1:
        sbest_m_idx = None


    out = []
    for i, e in enumerate(x):
        d = round(100 * e["D"], 1)
        m = round(e["M"], 1)
        if i == best_d_idx:
            d = "#best" + f"{d}"
        elif i == sbest_d_idx:
            d = "#sbest" + f"{d}"
        
        if i == best_m_idx:
            m = "#best" + f"{m}"
        elif i == sbest_m_idx:
            m = "#sbest" + f"{m}"
        out.append({
            "ablation": e["ablation"],
            "D": str(d),
            "M": str(m),
        })
    return out

In [18]:
def get_evaluations(task):
    evaluations = []
    norm = "none"
    steps = 10
    alpha = 1.0
    model = 16
    batch = 3
    dice_weight = 1.0
    ema_decay = 0.99

    temp = []
    for norm in ["none", "batch", "group"]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = norm
        temp.append(res)
    evaluations.extend(preprocess(temp))
    norm = "none"

    temp = []
    for steps in [5, 10, 20]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = steps
        temp.append(res)
    evaluations.extend(preprocess(temp))
    steps = 10

    temp = []
    for alpha in [1.0, 1.5, 2.0]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = alpha
        temp.append(res)
    evaluations.extend(preprocess(temp))
    alpha = 1.0

    temp = []
    for model in [12, 16, 20]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = model
        temp.append(res)
    evaluations.extend(preprocess(temp))
    model = 16

    temp = []
    for batch in [1, 3, 5]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = batch
        temp.append(res)
    evaluations.extend(preprocess(temp))
    batch = 3

    temp = []
    for dice_weight in [0.0, 1.0, 1.2, 1.5]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = dice_weight
        temp.append(res)
    evaluations.extend(preprocess(temp))
    dice_weight = 1.0

    temp = []
    for ema_decay in [0.0, 0.9, 0.99]:
        res = get_results(task, norm, steps, alpha, model, batch, dice_weight, ema_decay)
        res["ablation"] = ema_decay
        temp.append(res)
    evaluations.extend(preprocess(temp))
    ema_decay = 0.99
    return pd.DataFrame(evaluations)

In [19]:
#task = "prostate"
#task = "peso"
task = "cholec"

In [20]:
df_peso = get_evaluations("peso")
df_prostate = get_evaluations("prostate")
df_cholec = get_evaluations("cholec")

assert np.all(df_peso['ablation'] == df_prostate['ablation'])
assert np.all(df_peso['ablation'] == df_cholec['ablation'])

dfs = [df_prostate, df_peso, df_cholec]
dfs = [df.drop(columns=["ablation"]) for df in dfs]
df = pd.concat(dfs, keys=["prostate", "peso", "cholec"], axis=1)

In [21]:
print(df)

      prostate                   peso                 cholec           
             D          M           D          M           D          M
0    #best79.2   #best2.2  #sbest86.3   #best2.3        56.4   #best7.2
1   #sbest78.1        3.2        84.5        3.3   #best63.7       10.1
2         66.7        3.2   #best86.5        3.3  #sbest56.8       10.1
3    #best80.0   #best1.3        85.6   #best1.3        55.3   #best4.2
4         79.2  #sbest2.2  #sbest86.3  #sbest2.3  #sbest56.4  #sbest7.2
5   #sbest79.8        4.2   #best86.6        4.2   #best58.5       13.1
6         79.2   #best2.2  #sbest86.3        2.3   #best56.4   #best7.2
7    #best82.3  #sbest2.3        86.2   #best2.2  #sbest55.1  #sbest7.2
8   #sbest80.3        2.3   #best86.3  #sbest2.2        54.4        7.3
9         76.3   #best1.7        85.8   #best1.7        53.0   #best5.6
10   #best79.2  #sbest2.2  #sbest86.3  #sbest2.3  #sbest56.4  #sbest7.2
11  #sbest78.4        3.0   #best86.6        3.1   #best64.1    

In [22]:
df.to_latex("tables/ablation.tex", escape=False, index=False)